# Libraries and Packages

In [44]:
import gensim.corpora as corpora
import pandas as pd

from bertopic import BERTopic
from gensim.models.coherencemodel import CoherenceModel

# Data

In [2]:
file_path = "/Users/catherinepollack/Documents/dartmouth/research/aim3_facebook_covid19_obesity/data"
dat_obesity = "/220120_feature_matrix_no_bert.csv"

dat_obesity = pd.read_csv(str(file_path + dat_obesity))

/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (361,363,364,366,367,375,379,384,389,394,399,400,401,402) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [28]:
dat_obesity["date"] = pd.to_datetime(dat_obesity["Post Created Date"], errors = 'coerce')
dat_obesity_restricted = dat_obesity[(dat_obesity['date'] > '2020-01-06') & (dat_obesity['date'] < '2020-10-16')]

In [31]:
dat_obesity_restricted = dat_obesity_restricted.reset_index(drop = True)

In [33]:
dat_obesity_restricted.processed_text_bert = dat_obesity_restricted.processed_text_bert.fillna('')
topic_model = BERTopic(verbose = True, min_topic_size = 100)
topics = topic_model.fit_transform(dat_obesity_restricted.processed_text_bert)

Batches: 100%|██████████| 5477/5477 [11:14:31<00:00,  7.39s/it]     
2022-03-25 01:13:39,452 - BERTopic - Transformed documents to Embeddings
2022-03-25 01:22:36,230 - BERTopic - Reduced dimensionality with UMAP
2022-03-25 01:23:26,748 - BERTopic - Clustered UMAP embeddings with HDBSCAN


In [34]:
dat_obesity_restricted["topics"] = topics[0]

In [38]:
dat_obesity_restricted.to_csv(file_path + "/220325_obesity_with_topics.csv")
topic_model.save("220401_bert_topic_model_facebook")

In [37]:
topic_model.get_topic_info()[0:16]

,Topic,Count,Name
0,-1,87470,-1_the_to_and_of
1,0,3849,0_covid19_coronavirus_risk_obesity
2,1,2443,1_children_childhood_kids_child
3,2,2425,2_sugar_drinks_added_soda
4,3,2413,3_bariatric_surgery_bariatricsurgery_bariatrics
5,4,2090,4_my_me_was_myself
6,5,1807,5_sexual_occurs_intercourse_ejaculation
7,6,1668,6_she_owner_cat_her
8,7,1494,7_interesting_read_this_wow
9,8,1355,8_cancer_breast_colorectal_risk


In [47]:
for i in range(0,13):
    print(i)
    docs = topic_model.get_representative_docs()[i]
    for d in docs:
        print(d)
        print("-----")
    print("**********")

0
"In two cohorts of Chinese adults with COVID-19, those with obesity were at least three times more likely to have a severe case of the disease than those with normal weight, according to two studies published in Diabetes Care." Read the full story at https://www.healio.com/endocrinology/obesity/news/online/%7B45e16aa9-6309-4fff-a22c-d6f706156dce%7D/obesity-triples-odds-of-more-severe-symptoms-with-covid-19:=:https://www.healio.com/news/endocrinology/20200520/obesity-triples-odds-of-more-severe-symptoms-with-covid19
-----
Check out the latest research on Check out the latest research on "COVID-19 Rise in Younger Adults with Obesity: Visceral Adiposity Can Predict the Risk"— Iacobellis et. al. Read more in #ObesityJournal. #TOSCOVID-19 https://onlinelibrary.wiley.com/doi/10.1002/oby.22951:=:https://onlinelibrary.wiley.com/doi/10.1002/oby.22951?cookieSet=1
-----
Check out the latest research on "Obesity is Associated with Worst Outcomes in COVID-19: An Analysis of Early Data from New Yo

In [41]:
topic_model.visualize_hierarchy(topics = [0, 1, 2, 3, 4, 7, 8, 10, 11, 12])

In [ ]:
documents = pd.DataFrame({"Document": dat_obesity_restricted.processed_text_bert,
                          "ID":range(len(dat_obesity_restricted.processed_text_bert)),
                          "Topic":  topics[0]})

documents_per_topic = documents.groupby(['Topic'], as_index=False).agg({'Document': ' '.join})
cleaned_docs = topic_model._preprocess_text(documents_per_topic.Document.values)

vectorizer = topic_model.vectorizer_model
analyzer = vectorizer.build_analyzer()

words = vectorizer.get_feature_names()
tokens = [analyzer(doc) for doc in cleaned_docs]

In [45]:
dictionary = corpora.Dictionary(tokens)

corpus = [dictionary.doc2bow(token) for token in tokens]
topic_words = [[words for words, _ in topic_model.get_topic(topic)] 
               for topic in range(len(set(topics[0]))-1)]
coherence_model = CoherenceModel(topics=topic_words, 
                                 texts=tokens, 
                                 corpus=corpus,
                                 dictionary=dictionary, 
                                 coherence='c_v')
coherence = coherence_model.get_coherence()

In [46]:
coherence

0.7561582176383482